<a href="https://colab.research.google.com/github/Haebuk/recommendation/blob/main/goodbooks_10k_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os, sys, gc
from tqdm.notebook import tqdm

In [2]:
!git clone "https://github.com/Haebuk/recommendation.git"

fatal: destination path 'recommendation' already exists and is not an empty directory.


In [3]:
path = '/content/recommendation/goodbooks_10k/input/'

In [4]:
books = pd.read_csv(path + "books.csv")
book_tags = pd.read_csv(path + "book_tags.csv")
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
tags = pd.read_csv(path + "tags.csv")
to_read = pd.read_csv(path + "to_read.csv")

In [5]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()): 
    gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

## 한 사람당 200권의 책을 추천해주는 상황

In [6]:
rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

### Baseline
- 통계기반의 모델

In [7]:
books.sort_values(by='books_count', ascending=False)[0:5]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,Pride and Prejudice,eng,4.24,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...
1724,1725,6324090,6324090,55548884,3296,NaN,NaN,"Lewis Carroll, John Tenniel",1865.0,NaN,Alice's Adventures in Wonderland,eng,3.99,30426,85031,5047,1724,4863,18094,27848,32502,https://images.gr-assets.com/books/1391204048m...,https://images.gr-assets.com/books/1391204048s...
70,71,18490,18490,4836639,2618,141439475,9.780141e+12,"Mary Wollstonecraft Shelley, Percy Bysshe Shel...",1818.0,"Frankenstein; or, The Modern Prometheus",Frankenstein,eng,3.75,808589,883563,19993,29366,74142,231022,300495,248538,https://images.gr-assets.com/books/1381512375m...,https://images.gr-assets.com/books/1381512375s...
292,293,295,295,3077988,2573,753453800,9.780753e+12,Robert Louis Stevenson,1882.0,Treasure Island,Treasure Island,eng,3.82,268012,310515,7917,5349,19127,86487,114968,84584,https://images.gr-assets.com/books/1485248909m...,https://images.gr-assets.com/books/1485248909s...
42,43,10210,10210,2977639,2568,142437204,9.780142e+12,"Charlotte Brontë, Michael Mason",1847.0,Jane Eyre,Jane Eyre,eng,4.10,1198557,1286135,31212,35132,64274,212294,400214,574221,https://images.gr-assets.com/books/1327867269m...,https://images.gr-assets.com/books/1327867269s...


In [10]:
# 가장 많이 읽은 책 500권 추출
popular_rec_model = books.sort_values(by='books_count', ascending=False)['book_id'].values[0:500]

In [9]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    for rec in popular_rec_model[0:200]: 
        rec_list.append(rec)
    total_rec_list[user] = rec_list

In [14]:
import six # 파이썬 2, 3 compatibility
import math

# https://github.com/kakao-arena/brunch-article-recommendation/blob/master/evaluate.py

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt
        self.topn = topn

    def _ndcg(self): # NDCG 측정
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue
            
            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q

    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))


In [15]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.00015515720718927404
NDCG@200: 0.001811561666747593
EntDiv@200: 5.298317366548021


### Baseline 응용
- 많이 글은 글중에서도 평점이 높은 글들을 우선적으로 추천
- 내가 좋아하는 작가의 글을 우선적으로 추천
- 장바구니에 담긴 글과 작가의 글을 우선적으로 추천
- 읽은 글의 시리즈글이 나오면 추천 (해리포터 마법사의 돌 -> 비밀의 방)
- 최신의 글을 추천

In [16]:
train = pd.merge(train, books[['book_id', 'authors', 'ratings_count']], how='left', on='book_id')

In [17]:
agg = train.groupby(['user_id','authors'])['authors'].agg({'count'}).reset_index()
agg = agg.sort_values(by='count', ascending=False)
agg.head()

,user_id,authors,count
10937,17566,Bill Bryson,7
13509,21713,Bill Bryson,7
7377,11927,Bill Bryson,6
20746,33697,Bill Bryson,6
29112,49289,J.R.R. Tolkien,6


user 17566은 Bill Bryson의 책을 7권 읽음

이 유저에게 같은 작가의 책을 추천해주면 성능이 향상 될 수 있다.

In [27]:
author_books = books[['book_id', 'authors', 'ratings_count']].sort_values(by=['authors', 'ratings_count'], ascending=[True, False])
author_books = author_books.reset_index(drop=True)

author_books[author_books['authors']=='Bill Bryson']

,book_id,authors,ratings_count
944,9791,Bill Bryson,250192
945,21,Bill Bryson,191535
946,28,Bill Bryson,66947
947,24,Bill Bryson,58688
948,7507825,Bill Bryson,55296
949,25,Bill Bryson,44610
950,10538,Bill Bryson,42680
951,27,Bill Bryson,42170
952,26,Bill Bryson,40254
953,17262366,Bill Bryson,27237


In [19]:
author_rec_model = agg.merge(author_books, how='left', on=['authors'])

In [25]:
author_rec_model.head(20)

,user_id,authors,count,book_id,ratings_count
0,17566,Bill Bryson,7,9791,250192
1,17566,Bill Bryson,7,21,191535
2,17566,Bill Bryson,7,28,66947
3,17566,Bill Bryson,7,24,58688
4,17566,Bill Bryson,7,7507825,55296
5,17566,Bill Bryson,7,25,44610
6,17566,Bill Bryson,7,10538,42680
7,17566,Bill Bryson,7,27,42170
8,17566,Bill Bryson,7,26,40254
9,17566,Bill Bryson,7,17262366,27237


- 작가가 쓴 책마다 인기도는 다르기 떄문에 많이 읽은 책 순으로 정렬
- author 을 기준으로 합쳤기 때문에 user_id나 count는 위 데이터프레임에서 유의미한 값이 아님. author, book_id, rating_count만 보면됨
- agg가 author count로 정렬이 되었기 때문에 같은 작가의 책을 여러권 읽었으면 추천 정확도가 높아질 것

In [26]:
author_rec_model[author_rec_model['user_id'] == 17566]['book_id'].values

array([    9791,       21,       28,       24,  7507825,       25,
          10538,       27,       26, 17262366,   135611,       29,
       25489012,    10541,      431,      830,      816,      827,
        2845024, 22816087, 10552338,      823,      822,   116257,
            825,     1103,  5960325,  9500416,    24472, 18404427,
           3682,    51428,   127459,  7728889,  9464733,  6512140,
          13145,    13148,    13137,    13152,     7494,    13139,
          80568,    33676,  3393124,  1829709,    13153,    13140,
        6431790,    13143,    79378,    13128,    79379,    13162,
        7201913,     6588,    84736,   552887,    53625,    13151,
         110444, 11946245,  6405929,  1713912,     5575,  7856305,
       10343389, 13521299, 13097901, 17333277, 20800147, 25066591,
              3,    15881,        6,   136251,        2,        1,
          99298,        8,      122,     7354,     1627,    27999])

In [23]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    for rec in author_rec_model_: 
        rec_list.append(rec)
    
    if len(rec_list) < 200: # 200권 보다 작은 경우 인기있는 책 추천
        for i in popular_rec_model[0:200]:
            rec_list.append(rec)
        
    total_rec_list[user] = rec_list[0:200]

In [28]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.01488745219381199
NDCG@200: 0.004724455575031803
EntDiv@200: 5.860416089913054


- 기본 통계 모델
    - MAP@200: 0.00015515720718927404
    - NDCG@200: 0.001811561666747593
    - EntDiv@200: 5.298317366548021
- 선호 작가 추가 모델
    - MAP@200: 0.01488745219381199
    - NDCG@200: 0.004724455575031803
    - EntDiv@200: 5.860416089913054

MAP가 약 140배 증가!!

### 후처리
- 내가 읽은 책은 추천해주면 안됨
- 내가 읽은 언어와 맞는 책 추 

In [29]:
# 내가 읽은 책의 목록을 추출 
read_list = train.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
read_list.head()

,user_id,unique
0,1,[6285]
1,2,[8034]
2,3,[9014]
3,4,[8464]
4,5,"[6646, 4829]"


In [30]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    seen = read_list[read_list['user_id'] == user]['unique'].values[0]
    for rec in author_rec_model_: # 읽지 않았을 경우 리스트에 추가
        if rec not in seen:
            rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            if rec not in seen:
                rec_list.append(rec)

    total_rec_list[user] = rec_list[0:200]

In [31]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.01616279618994572
NDCG@200: 0.0052927872962257785
EntDiv@200: 5.10111607321712


- 기본 통계 모델
    - MAP@200: 0.00015515720718927404
    - NDCG@200: 0.001811561666747593
    - EntDiv@200: 5.298317366548021
- 선호 작가 추가 모델
    - MAP@200: 0.01488745219381199
    - NDCG@200: 0.004724455575031803
    - EntDiv@200: 5.860416089913054
- 읽은 책 후처리 모델
    - MAP@200: 0.01616279618994572
    - NDCG@200: 0.0052927872962257785
    - EntDiv@200: 5.10111607321712

MAP. NDCG 소폭 증가

In [32]:
# 내가 읽을 수 있는 언어의 목록을 추출 
## User에 대한 메타정보가 있으면 쉽게 추출가능하지만, 현재는 없으므로 직접 생성 
## Ratings에서 읽은 책들의 언어를 전부 수집해서 해당 언어의 책들을 가능한 언어로 설정 
language = pd.merge(train, books[['book_id', 'language_code']], how='left', on='book_id')

In [33]:
language_list = language.groupby(['user_id'])['language_code'].agg({'unique'}).reset_index()
language_list.head()

,user_id,unique
0,1,[nan]
1,2,[nan]
2,3,[eng]
3,4,[nan]
4,5,[nan]
